# VencoPy Tutorial 1

This tutorial showcases the general structure and workflow of VencoPy, as well as some basic features of its 4 main classes:
- dataParsers
- tripDiaryBuilder
- gridModeler
- flexEstimator

## Setting up the working space

In [3]:
import os, sys
import pandas as pd
import numpy as np
import yaml
import pathlib

sys.path.append("../../classes")
sys.path.append("../../../vencopy")

from dataParsers import DataParser
from tripDiaryBuilders import TripDiaryBuilder
from gridModelers import GridModeler
from flexEstimators import FlexEstimator

In [8]:
datasetID = 'MiD17'

pathGlobalConfig = pathlib.Path.cwd().parent.parent / 'config' / 'globalConfig.yaml'
with open(pathGlobalConfig) as ipf:
    globalConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
    
pathLocalPathConfig = pathlib.Path.cwd().parent.parent  / 'config' / 'localPathConfig.yaml'
with open(pathLocalPathConfig) as ipf:
    localPathConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathParseConfig = pathlib.Path.cwd().parent.parent  / 'config' / 'parseConfig.yaml'
with open(pathParseConfig) as ipf:
    parseConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathTripConfig = pathlib.Path.cwd().parent.parent  / 'config' / 'tripConfig.yaml'
with open(pathTripConfig) as ipf:
    tripConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathGridConfig = pathlib.Path.cwd().parent.parent  / 'config' / 'gridConfig.yaml'
with open(pathGridConfig) as ipf:
    gridConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathEvaluatorConfig = pathlib.Path.cwd().parent.parent  / 'config' / 'evaluatorConfig.yaml'
with open(pathEvaluatorConfig) as ipf:
    evaluatorConfig = yaml.load(ipf, Loader=yaml.SafeLoader)
pathFlexConfig = pathlib.Path.cwd().parent.parent  / 'config' / 'flexConfig.yaml'
with open(pathFlexConfig) as ipf:
    flexConfig = yaml.load(ipf, Loader=yaml.SafeLoader)

Parsing properties set up
Starting to retrieve local data file from C:\8_Work\Data\MiD17\B2_Regional-DatensatzpaketEncrypted\STATA\MiD2017_Regional_Wege.dta
Finished loading 960619 rows of raw data of type .dta
Finished harmonization of variables
Starting filtering, applying 8 filters.
The following values were taken into account after filtering:
{'isMIVDriver': 577155,
 'tripDistance': 892627,
 'tripEndClock': 960619,
 'tripEndHour': 896186,
 'tripIsIntermodal': 790421,
 'tripPurpose': 958025,
 'tripStartClock': 960619,
 'tripStartHour': 898923}
All filters combined yielded a total of 414683 was taken into account
This corresponds to 43.16831126596497 percent of the original data
Parsing completed


In [11]:
# Trip distance and purpose diary compositions
vpTripDiary = TripDiaryBuilder(datasetID=datasetID, tripConfig=tripConfig, globalConfig=globalConfig, ParseData=vpData)

## _dataParser_ class

In [ ]:
vpData = DataParser(datasetID=datasetID, parseConfig=parseConfig, globalConfig=globalConfig, localPathConfig=localPathConfig, loadEncrypted=False)

## _tripDiaryBuilder_ class

In [ ]:
# Evaluate drive and trip purpose profiles
vpEval = Evaluator(globalConfig=globalConfig, evaluatorConfig=evaluatorConfig,
                   parseData=pd.Series(data=vpData, index=[datasetID]), label='SESPaperTest')
vpEval.hourlyAggregates = vpEval.calcVariableSpecAggregates(by=['tripStartWeekday'])
vpEval.plotAggregates()

## _gridModeler_ class

In [ ]:
# Grid model applications
vpGrid = GridModeler(gridConfig=gridConfig, globalConfig=globalConfig, datasetID=datasetID)
vpGrid.assignSimpleGridViaPurposes()
vpGrid.writeOutGridAvailability()

## _flexEstimator_ class

In [ ]:
# Estimate charging flexibility based on driving profiles and charge connection
vpFlex = FlexEstimator(flexConfig=flexConfig, globalConfig=globalConfig, evaluatorConfig=evaluatorConfig, datasetID=datasetID, ParseData=vpData)
vpFlex.baseProfileCalculation()
vpFlex.filter()
vpFlex.aggregate()
vpFlex.correct()
vpFlex.normalize()
vpFlex.writeOut()

vpEval.plotProfiles(flexEstimator=vpFlex)